## Tarea 3 - Métodos de Aprendizajes de Máquinas en Data Science

### Integrantes: Sofía Álvarez, Sebastián Dinator.

### Importación de librerías y datos

In [20]:
import sys
!{sys.executable} -m pip install numpy pandas emoji

In [21]:
import pandas as pd
import numpy as np
import re
import string

In [22]:
# Se cargan los datos
train_data = pd.read_csv('train_tweets.csv') # Entrenamiento
eval_data = pd.read_csv('test_tweets.csv') # Prueba

### Limpieza de datos

In [23]:
# Función para limpiar los tweets
import emoji

train_data['text'] = train_data['text'].apply(lambda s: emoji.replace_emoji(s, ''))
eval_data['text'] = eval_data['text'].apply(lambda s: emoji.replace_emoji(s, ''))

def clean_tweet(tweet):
    tweet = tweet.lower() # Convertir a minúsculas (dejar??)
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE) # Eliminar URLs
    tweet = re.sub(r'@\w+', '', tweet) # Eliminar menciones de usuario
    tweet = re.sub(r'#\w+', '', tweet) # Eliminar hashtags
    tweet = re.sub(r'\d+', '', tweet) # Eliminar números
    tweet = tweet.translate(str.maketrans('', '', string.punctuation)) # Eliminar puntuaciones
    tweet = re.sub(r'\\r\\n|\\n|\\r', ' ', tweet) # Eliminar caracteres especiales
    tweet = re.sub(r'\s+', ' ', tweet).strip() # Eliminar espacios adicionales
    # tweet = tweet.apply(lambda s: emoji.replace_emoji(s, ''))
    return tweet

In [24]:
# Imprimir los nombres de las columnas
print("Columnas en train_data:", train_data.columns)
print("Columnas en test_data:", eval_data.columns)

Columnas en train_data: Index(['tweet_id', 'text', 'Odio'], dtype='object')
Columnas en test_data: Index(['tweet_id', 'text'], dtype='object')


In [25]:
# Limpieza a los conjuntos de datos
train_data['cleaned_text'] = train_data['text'].apply(clean_tweet)
eval_data['cleaned_text'] = eval_data['text'].apply(clean_tweet)

print(train_data.head())
print(eval_data.head())

              tweet_id                                               text  \
0  1399516036240662528  En una amistad o soy tipo: \r\nLo peor es que ...   
1  1320788179721560065  QUIEN FUE LA MARACA CULIA TE VOY A MATAR PERRA...   
2  1079889645280944129  menos mal q se recibe el año con ropa blanca j...   
3  1369254390134145033  Cuantos INFILTRADOS extranjeros hay ahi https:...   
4  1533854824378290176  #ENCONTRADO #GUAYAQUIL\r\nFecha: 06/06/22\r\nS...   

   Odio                                       cleaned_text  
0     0  en una amistad o soy tipo lo peor es que siemp...  
1     1  quien fue la maraca culia te voy a matar perra...  
2     0  menos mal q se recibe el año con ropa blanca j...  
3     1            cuantos infiltrados extranjeros hay ahi  
4     0  fecha sector norte autopista a la altura del p...  
              tweet_id                                               text  \
0  1533854540763742209  Estoy de acuerdo que corrijas a tus hijos pero...   
1  12777565045197

## Naive Bayes

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

X_train = train_data['cleaned_text']
y_train = train_data['Odio']
X_test = eval_data['cleaned_text']

vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

NB = MultinomialNB(alpha=10)
NB.fit(X_train_vect, y_train)

predicted = NB.predict(X_test_vect)
print("Predicciones:\n", predicted)


Predicciones:
 [1 1 1 ... 1 1 1]


In [27]:
import pandas as pd

predicted = NB.predict(X_test_vect)
predictions_df = pd.DataFrame(predicted, columns=['Predicción_Odio'])
eval_data_with_predictions = eval_data.join(predictions_df)
eval_data_with_predictions.to_csv('test_tweets_with_predictions.csv', index=False)
